<a href="https://colab.research.google.com/github/komazawa-deep-learning/komazawa-deep-learning.github.io/blob/master/2025notebooks/2025_0213Gloq_API_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 0. Google ドライブに保存

必ず、この Colab ノートブックを「ドライブにコピー」してから使う。


# 1. Groq の API を取得

[https://console.groq.com/keys](https://console.groq.com/keys)

参考

* [https://console.groq.com/docs/quickstart](https://console.groq.com/docs/quickstart)
* [https://note.com/synth_brain/n/ne8a5f1924455](https://note.com/synth_brain/n/ne8a5f1924455)


# 2. 必要ライブラリのインストール

In [ ]:
try:
    import groq
except ImportError:
    !pip install --upgrade groq

# APIを使ったLlamaの利用

API = "自分のAPI-Keyをコピペする"

ダブルクオーテーションは必要。

Keyのみをコピペで入れ替える。

In [ ]:
API = "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"
API

'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'

## サンプルと同じコード

https://console.groq.com/docs/quickstart

"content": "***ここを自分でいろいろ変えてみる。***"

In [ ]:
from groq import Groq

client = Groq(
    api_key=API,
)

chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "人生で最も重要なことは何かを日本語で説明してください",
#            "content": "Explain the importance of natural language processing.",
        }
    ],
    model="llama-3.3-70b-versatile",
)

print(chat_completion.choices[0].message.content)

In [ ]:
for k,v in chat_completion.model_fields.items():
    print(k,v)

In [ ]:
text = 'Please show us a code to get an embedding vector of this sentence.'
text = 'この文の埋め込みベクトルを得る python コードを教えて下さい'

chat_completion = client.chat.completions.create(
    messages=[
        # {
        #     "role": "system",
        #     "content": "Please transrate your responce in Japanese.And answer only Janapse responce.",
        # },
        {
            "role": "user",
            "content": text,
        }
    ],
    model="mixtral-8x7b-32768",
    #model="llama-3.3-70b-versatile",
)

print(chat_completion.choices[0].message.content)

この文の埋め込みベクトルを得るためには、Natural Language Processing (NLP) のライブラリを使用します。たとえば、TensorFlow のTextVectorization または gensim のWord2Vec、GloVeなどです。

以下はTextVectorizationを使った例です。

```python
import tensorflow as tf

# 単語を1と0のベクターに変換するためのTextVectorization
vectorizer = tf.keras.layers.TextVectorization()
vectorizer.adapt(["この", "文", "の", "埋め込み", "ベクトル"])

# 文章をベクトルに変換
vector = vectorizer(["この文の埋め込みベクトル"])

# 出力
print(vector[0])
```

ただし、この例では単語が1と0のベクトルに変換されます。埋め込みベクトルを取得するためには、事前に学習された埋め込みモデルを使用する必要があります。例えば、gensimのWord2Vecで事前に学習されたモデルを使用して1文を埋め込みベクトルに変換することはできます。以下はその例です。

```python
import gensim.downloader as api
model = api.load("word2vec-google-news-300")

# 埋め込みベクトルを取得する
vector = model.wv.get_vector("この") + model.wv.get_vector("文") + model.wv.get_vector("の") + model.wv.get_vector("埋め込み") + model.wv.get_vector("ベクトル")

# 出力
print(vector)
```

 note: TensorFlow 2.x を使用しています。もしそれ以前のバージョンを使用している場合は少し違うコードになります。


In [ ]:
#print(dir(chat_completion))
print(chat_completion.model_fields)
#print(dir(chat_completion.choices))
#print(chat_completion.choices)
#help(chat_completion.choices[0])
#chat_completion.choices[0].to_dict()

{'id': FieldInfo(annotation=str, required=True), 'choices': FieldInfo(annotation=List[Choice], required=True), 'created': FieldInfo(annotation=int, required=True), 'model': FieldInfo(annotation=str, required=True), 'object': FieldInfo(annotation=Literal['chat.completion'], required=True), 'system_fingerprint': FieldInfo(annotation=Union[str, NoneType], required=False, default=None), 'usage': FieldInfo(annotation=Union[CompletionUsage, NoneType], required=False, default=None)}


## チャットボット

｢この文章をトークン化してください｣。をトークン化して，そのトークンIDを教えて下さい。

In [ ]:
from groq import Groq

client = Groq(api_key=API)

messages = [
    {
        "role": "system",
        "content": "You're a chatbot. Please have a chat with me. When I type in Japanese, please answer in Japanese.",
    }
]

print("Chatbot: Hello! You can start chatting. Type 'exit' to end the conversation.")

while True:
    user_input = input("You: ")  # ユーザー入力を取得

    if user_input.lower() in ["exit", "quit"]:
        print("Chatbot: Goodbye!")
        break

    # ユーザーのメッセージを履歴に追加
    messages.append({"role": "user", "content": user_input})

    # API にリクエストを送信
    chat_completion = client.chat.completions.create(
        messages=messages,
        model="llama-3.3-70b-versatile",
    )

    # 応答を取得して表示
    response = chat_completion.choices[0].message.content
    print("Chatbot:", response)

    # 応答を履歴に追加
    messages.append({"role": "assistant", "content": response})


### 終了するためには「exit」や「quit」と入力してください ###

## プロンプトを先に書いておくパターン

以下ではサンプルとして、入力された英文のエラーを訂正するものにしている。

prompt = "Reply with a corrected version of the input sentence with all grammatical, spelling, and punctuation errors fixed. Be strict about the possible errors. If there are no errors, reply with a copy of the original sentence. Please do not add any unnecessary explanations."

text = "In additoin to making money by themselves can increase the sense of independece from their parents because they can earn money by working hard."

In [ ]:
prompt = "Reply with a corrected version of the input sentence with all grammatical, spelling, and punctuation errors fixed. Be strict about the possible errors. If there are no errors, reply with a copy of the original sentence. Please do not add any unnecessary explanations."

In [ ]:
prompt

In [ ]:
text = "In additoin to making money by themselves can increase the sense of independece from their parents because they can earn money by working hard."

In [ ]:
# text = input('Text:')

In [ ]:
text

In [ ]:
from groq import Groq

client = Groq(
    api_key=API,
)
completion = client.chat.completions.create(
    model="llama-3.3-70b-versatile",
    messages=[
        {
            "role": "system",
            "content": prompt,
        },
        {
            "role": "user",
            "content": text,
        }
    ],
    temperature=1,  # 1が最大（OpenAIは2）、0が最小、低い値を指定すると回答のランダム性を減らす
    max_tokens=1024,
    top_p=1, # 低い値を指定すると、可能性の低い選択肢を無視。0 or 1で無効になる。
    # top_p と temperature は共に出力の多様性を制御するパラメータだが、同時に両方を変更することは推奨されていない
    stream=True, # APIの応答を一度にすべて受け取るのではなく、小さなチャンク（部分）ごとに受け取りながら処理
    stop=None, # 生成を停止する条件（やキーワード）を指定（改行2回までなど）
)

for chunk in completion:
    print(chunk.choices[0].delta.content or "", end="")

[difff](https://difff.jp/)